# Data Cleaning and Manipulation

In [ ]:
import pandas as pd

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.width', None)  # Adjust the width to display everything
pd.set_option('display.max_colwidth', None)  # Show full column content

import datetime as dt

import fasttext
import re
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import numpy as np

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize


In [ ]:
# read csv files
biden_df = pd.read_csv(r"C:\Users\User\iCloudDrive\Cursos\Data Circle\DataCircle_Twitter_Project\hashtag_joebiden.csv",lineterminator='\n')
trump_df = pd.read_csv(r"C:\Users\User\iCloudDrive\Cursos\Data Circle\DataCircle_Twitter_Project\hashtag_donaldtrump.csv", lineterminator='\n')

print(biden_df.info())
print(trump_df.info())

## Data Preprocessing

In [ ]:
# create a new column 'candidate' to differentiate tweets of each candidate after concatenation 
trump_df['candidate'] = 'trump'

# biden dataframe 
biden_df['candidate'] = 'biden'

# combining the dataframes 
twitter_df = pd.concat([trump_df, biden_df])


In [ ]:
twitter_df

In [ ]:
twitter_df.info()

In [ ]:
twitter_df.isnull().sum()

### Remove irrelevant data

In [ ]:
# Remove irrelevant data ("user_name", "user_screen_name", "user_description")
twitter_df = twitter_df[['created_at', 'tweet_id', 'tweet', 'likes', 'retweet_count', 'source',
       'user_id', 'user_join_date', 'user_followers_count', 'user_location', 'city', 'country', 'state', 'candidate']]

### Drop duplicate values

In [ ]:
# Drop duplicates based on 'tweet_id', 'tweet', and 'created_at'
twitter_df = twitter_df[~(twitter_df.duplicated(subset=['tweet_id', 'tweet', 'created_at'], keep=False))]

### Ensure data consistency

In [ ]:
# Convert date columns to datetime
columns_to_datetime = ['created_at', 'user_join_date']
twitter_df[columns_to_datetime] = twitter_df[columns_to_datetime].apply(pd.to_datetime, errors='coerce')

In [ ]:
# Function to convert selected columns to int
def convert_columns_to_int(df, columns):
    for column in columns:
        df[column] = pd.to_numeric(df[column], errors='coerce').astype('int64') 
    return df

columns_to_convert_int = ['likes', 'retweet_count', 'user_followers_count', 'tweet_id', "user_id"]
twitter_df = convert_columns_to_int(twitter_df, columns_to_convert_int)

In [ ]:
twitter_df.info()

In [ ]:
twitter_df

In [ ]:
# Function to clean the tweet column
def clean_tweet_column(df, tweet_column):

    # Work on a copy of the tweet column to avoid modifying the original
    tweet_cleaned = df[tweet_column].copy()

    # Clean and convert to lowercase
    tweet_cleaned = tweet_cleaned.str.lower().str.strip()

    # Replace '\n' with space ' '
    tweet_cleaned = tweet_cleaned.str.replace('\n', ' ')

    # Remove URLs
    url_pattern = r'http[s]?://\S+|www\.\S+'
    tweet_cleaned = tweet_cleaned.str.replace(url_pattern, '', regex=True)

    # Remove unwanted symbols (keeping letters, numbers, hashtags, and spaces)
    tweet_cleaned = tweet_cleaned.str.replace(r'[^a-zA-Z0-9# ]', '', regex=True)

    return tweet_cleaned

# Apply the function and assign the cleaned result to 'tweet_cleaned' without modifying 'tweet'
twitter_df["tweet_cleaned"] = clean_tweet_column(twitter_df, 'tweet')


In [ ]:
# Ensure all text columns are lowercase for consistent NLP analysis.
def clean_and_convert_text_columns(df, text_columns):
    # Clean and convert text columns to lowercase
    for column in text_columns:
        df[column] = df[column].str.lower().str.strip()  # Convert to lowercase, remove leading and trailing spaces

    return df

# Return the names of object columns
text_columns_to_convert = ["source", "user_location", "city", "state", "country"]
twitter_df = clean_and_convert_text_columns(twitter_df, text_columns_to_convert)

In [ ]:
twitter_df["country"].value_counts()

In [ ]:
# Create a mapping dictionary
country_mapping = {
    'united states of america': 'united states',
    'the netherlands': 'netherlands'
    }

# Standardize country names using the mapping
twitter_df['country'] = twitter_df['country'].replace(country_mapping)

### Handle missing values

In [ ]:
twitter_df.isnull().sum()

In [ ]:
# Fill missing values in the 'source' column with the most frequent value (mode)
twitter_df["source"] = twitter_df["source"].fillna(twitter_df["source"].mode()[0])

In [ ]:
# Filter rows where user location is null and either lat, long, city, state or country is not null
twitter_df[(twitter_df["user_location"].isna()) & (~(twitter_df["city"].isna()) | ~(twitter_df["state"].isna()) | ~(twitter_df["country"].isna()))]

Based on the previous output, whenever the user location is null, lat, long, city, state and country will also be null. Which means user location column can not be filled with the other location columns

In [ ]:
# Fill null location columns with "unknown"
twitter_df["user_location"] = twitter_df["user_location"].fillna("unkown")
twitter_df["city"] = twitter_df["city"].fillna("unkown")
twitter_df["state"] = twitter_df["state"].fillna("unkown")
twitter_df["country"] = twitter_df["country"].fillna("unkown")

In [ ]:
twitter_df.isnull().sum()

In [ ]:
twitter_df.shape

### Detect tweet language and filter only english tweets

In [ ]:
# Load the FastText pre-trained language identification model
model = fasttext.load_model('lid.176.bin')

# Function to detect language
def detect_language(text):
    try:
        predictions = model.predict(text, k=1)  # Get top 1 language prediction
        lang_code = predictions[0][0].split("__label__")[1]  # Extract language code
        return lang_code
    except Exception as e:
        return 'unknown'  # Return 'unknown' for any errors

# Apply language detection to your "tweet" column 
twitter_df['language'] = twitter_df['tweet_cleaned'].apply(detect_language)

In [ ]:
twitter_df['language'].value_counts()

In [ ]:
twitter_df['language'].unique()

In [ ]:
# filter only twitter in english language
twitter_df = twitter_df[twitter_df['language']=='en']

In [ ]:
# Drop language column
twitter_df = twitter_df.drop(columns='language')

In [ ]:
twitter_df.info()

### Remove StopWords

In [ ]:
# Get the list of stop words in English
stop_words = set(stopwords.words('english'))

# Define a function to remove stop words
def remove_stopwords(text):
     # Split the text into words by spaces, filter out stop words, and rejoin
    filtered_words = [word for word in text.split() if word not in stop_words]
    return ' '.join(filtered_words)

# Apply the function to the 'tweet' column
twitter_df['tweet_cleaned'] = twitter_df['tweet_cleaned'].apply(remove_stopwords)

In [ ]:
twitter_df[["tweet", "tweet_cleaned"]]

In [ ]:
# Drop original tweet column
twitter_df = twitter_df.drop(columns="tweet")

In [ ]:
twitter_df.info()

### Handle Outliers

In [ ]:
twitter_df.describe()


In [ ]:
print("likes skewness:")
print(twitter_df["likes"].skew())
print("\n")
print("likes kurtosis:")
print(twitter_df["likes"].kurtosis())
print("\n")
print("retweet skewness:")
print(twitter_df["retweet_count"].skew())
print("\n")
print("retweet kurtosis:")
print(twitter_df["retweet_count"].kurtosis())

In [ ]:
# Subset the data for 'likes' and 'retweets'
likes = twitter_df['likes']
retweets = twitter_df['retweet_count']

# 1. Histogram and KDE for 'likes' and 'retweets'
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.histplot(likes, bins=50, kde=True)
plt.title('Likes Distribution')

plt.subplot(1, 2, 2)
sns.histplot(retweets, bins=50, kde=True)
plt.title('Retweets Distribution')

plt.show()

# 2. Boxplot for 'likes' and 'retweets' (helps identify outliers)
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.boxplot(x=likes)
plt.title('Boxplot of Likes')

plt.subplot(1, 2, 2)
sns.boxplot(x=retweets)
plt.title('Boxplot of Retweets')

plt.show()

In [ ]:
# Detect outliers using IQR for 'likes' and 'retweets'

# For 'likes'
Q1_likes = twitter_df['likes'].quantile(0.25)
Q3_likes = twitter_df['likes'].quantile(0.75)
IQR_likes = Q3_likes - Q1_likes

lower_bound_likes = Q1_likes - 3 * IQR_likes
upper_bound_likes = Q3_likes + 3 * IQR_likes

# For 'retweets'
Q1_retweets = twitter_df['retweet_count'].quantile(0.25)
Q3_retweets = twitter_df['retweet_count'].quantile(0.75)
IQR_retweets = Q3_retweets - Q1_retweets

lower_bound_retweets = Q1_retweets - 3 * IQR_retweets
upper_bound_retweets = Q3_retweets + 3 * IQR_retweets

# Identify outliers in 'likes' and 'retweets'
outliers_likes = twitter_df[(twitter_df['likes'] < lower_bound_likes) | (twitter_df['likes'] > upper_bound_likes)]
outliers_retweets = twitter_df[(twitter_df['retweet_count'] < lower_bound_retweets) | (twitter_df['retweet_count'] > upper_bound_retweets)]

print(f"Number of outliers in 'likes' using IQR: {len(outliers_likes)}")
print(f"Number of outliers in 'retweets' using IQR: {len(outliers_retweets)}")


In [ ]:
print(outliers_likes["likes"].describe())
print("\n")
print(outliers_retweets["retweet_count"].describe())

Observations:

Likes:
* The number of outliers decreased after using the 3x IQR factor, which means the bound have been tightened and fewer low outliers have been excluded .
* The mean, median, and standard deviation suggest that many of the outliers are still relatively small, but there are a few extreme values (as indicated by the high max of 165,702).

Retweets:
* The retweet outliers remain unchanged with the 3x IQR factor, indicating that the distribution of retweets may not be as heavily skewed at the lower end. 
* The presence of very low outliers (min = 1) suggests that many entries are being flagged due to the distribution's natural skew.

In [ ]:
twitter_df.to_csv("twitter_cleaned_data.csv", index=False)